In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

from statsmodels.sandbox.stats.runs import runstest_1samp
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.stattools import acf
from sklearn.decomposition import PCA


In [2]:
# Loading a CSV file into a DataFrame
df_void = pd.read_csv('../Processed dataset/df_voids.csv')
df_parts = pd.read_csv('../Processed dataset/df_parts.csv')
df_void_new = pd.read_csv('../Processed dataset/df_new_voids.csv')
df_parts_new = pd.read_csv('../Processed dataset/df_new_parts.csv')

#df_parts.head(10)
df_void_new.head(10)

,Image name,Position,Region type,ID,Area [pixels],Perimeter [pixels],Eccentricity,Orientation [radians],Solidity,Extent,Major Axis Length [pixels],Minor Axis Length [pixels],Equivalent Diameter [pixels],Part_ID,Num_voids,Num_Wpixels,Num_Bpixels,Wpixels_ratio
0,img01.bmp,top_left,void,1,104,40.142,0.752,-1.556,0.912,0.693,14.874,9.809,11.507,0,41,38291,13692,0.736606
1,img01.bmp,top_left,void,2,106,47.799,0.935,-1.335,0.898,0.589,20.548,7.293,11.617,0,41,38291,13692,0.736606
2,img01.bmp,top_left,void,3,263,80.355,0.939,-1.307,0.895,0.501,32.887,11.318,18.299,0,41,38291,13692,0.736606
3,img01.bmp,top_left,void,4,211,64.006,0.884,-1.478,0.887,0.601,25.354,11.839,16.391,0,41,38291,13692,0.736606
4,img01.bmp,top_left,void,5,159,56.355,0.874,0.932,0.888,0.523,21.442,10.408,14.228,0,41,38291,13692,0.736606
5,img01.bmp,top_left,void,6,211,73.077,0.951,-1.313,0.847,0.471,31.267,9.708,16.391,0,41,38291,13692,0.736606
6,img01.bmp,top_left,void,7,339,75.841,0.326,0.692,0.929,0.589,22.505,21.278,20.776,0,41,38291,13692,0.736606
7,img01.bmp,top_left,void,8,361,78.184,0.502,1.001,0.928,0.602,24.343,21.051,21.439,0,41,38291,13692,0.736606
8,img01.bmp,top_left,void,9,329,77.770,0.718,0.249,0.901,0.580,26.044,18.126,20.467,0,41,38291,13692,0.736606
9,img01.bmp,top_left,void,10,311,82.083,0.900,-0.839,0.899,0.461,31.733,13.860,19.899,0,41,38291,13692,0.736606


In [31]:
 # normality test df void for img = 
start_index = 4

list_variable = df_void_new.columns.tolist()
end_index = len(df_void_new.columns) 

for img in range (40):

    for col in df_void_new.columns[start_index:end_index]:
        dataset = df_void_new.loc[df_void_new['Part_ID'] == img, col].values  # In teoria importa l'ordine perchè dovrei confrontare buco per buco, posso usare comunque t-boostrap? 
# Verify the normality of the data
# Shapiro-Wilk test
        _, p_value_SW = stats.shapiro(dataset)
        print('p-value of the Shapiro-Wilk test for "{}": {:.3f}'.format(col, float(p_value_SW)))
        if (p_value_SW < 0.01 and col in list_variable):
            list_variable.remove(col)
    



p-value of the Shapiro-Wilk test for "Area [pixels]": 0.290
p-value of the Shapiro-Wilk test for "Perimeter [pixels]": 0.322
p-value of the Shapiro-Wilk test for "Eccentricity": 0.000
p-value of the Shapiro-Wilk test for "Orientation [radians]": 0.001
p-value of the Shapiro-Wilk test for "Solidity": 0.010
p-value of the Shapiro-Wilk test for "Extent": 0.027
p-value of the Shapiro-Wilk test for "Major Axis Length [pixels]": 0.795
p-value of the Shapiro-Wilk test for "Minor Axis Length [pixels]": 0.090
p-value of the Shapiro-Wilk test for "Equivalent Diameter [pixels]": 0.351
p-value of the Shapiro-Wilk test for "Part_ID": 1.000
p-value of the Shapiro-Wilk test for "Num_voids": 1.000
p-value of the Shapiro-Wilk test for "Num_Wpixels": 1.000
p-value of the Shapiro-Wilk test for "Num_Bpixels": 1.000
p-value of the Shapiro-Wilk test for "Wpixels_ratio": 1.000
p-value of the Shapiro-Wilk test for "Area [pixels]": 0.153
p-value of the Shapiro-Wilk test for "Perimeter [pixels]": 0.096
p-value 

In [32]:
list_variable

['Image name',
 'Position',
 'Region type',
 'ID',
 'Area [pixels]',
 'Major Axis Length [pixels]',
 'Part_ID',
 'Num_voids',
 'Num_Wpixels',
 'Num_Bpixels',
 'Wpixels_ratio']

In [27]:
# Not normal
# for each void we check the area --> SPC FOR THE AREA 